In [24]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
import pandas as pd
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)

In [47]:
auto = pd.read_csv('../../data_set/Auto.csv', na_values='?');

In [48]:
auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
393,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
394,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
395,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


In [52]:
auto = auto.dropna()

Perform a simple linear regression with `mpg` as the response and `horsepower` as the predictor.

The codes below generate predictor matrix `X`.

In [54]:
horsepower = MS(['horsepower']);
X = horsepower.fit_transform(auto);
X[:5]

,intercept,horsepower
0,1.0,130.0
1,1.0,165.0
2,1.0,150.0
3,1.0,150.0
4,1.0,140.0


In [55]:
y = auto['mpg']
y[:5]

0    18.0
1    15.0
2    18.0
3    16.0
4    17.0
Name: mpg, dtype: float64

In [56]:
y = auto['mpg'];
model = sm.OLS(y, X);
results = model.fit();
summarize(results)

,coef,std err,t,P>|t|
intercept,39.9359,0.717,55.660,0.0
horsepower,-0.1578,0.006,-24.489,0.0
